# Resumo Prova Final

## Aula 13 - Normalização

### Formas Normais

**Normalização** → processo de rearranjar o banco de dados para que obedeça às várias **formas normais**.

O processo contrário, ou seja, regredir o projeto do banco de dados para formas normais menores, é chamado de **desnormalização**.

##### Objetivo

- Evitar repetições
- Recuperar informações de forma fácil

#### 1NF - Primeira Forma Normal

Uma entidade está na primeira forma normal quando cada tupla tem apenas um valor por atributo

Exemplo:

<!-- tabela que nao esta na 1nf -->
| id_usuario (PK) | nome | id_endereço | endereço | telefone |
|-----------------|------|------------|----------|----------|
1 | Juca | 100 | Rua Quatá, 300, São Paulo/SP | 111-1111, 333-3333
2 | Maria | 200 | Rua Casa Do Ator, 537, Ap 42, São Paulo/SP | 222-2222
3 | Roberto | 300 | Av. Bias Fortes 382, Belo Horizonte/MG | 333-3333

Esta tabela não está na 1NF, pois as colunas `endereço` e `telefone` possuem mais de um valor.

Solução:

<!-- tabela que esta na 1nf -->
| id_usuario (PK) | nome | id_endereço | endereço | cidade | estado | telefone |
|-----------------|------|------------|----------|--------|-------|----------|
1 | Juca | 100 | Rua Quatá, 300 | São Paulo | SP | 111-1111
1 | Juca | 100 | Rua Quatá, 300 | São Paulo | SP | 333-3333
2 | Maria | 200 | Rua Casa Do Ator, 537, Ap 42 | São Paulo | SP | 222-2222
3 | Roberto | 300 | Av. Bias Fortes 382 | Belo Horizonte | MG | 333-3333

#### 2NF - Segunda Forma Normal

Uma entidade está na segunda forma normal quando obedecer à 1NF e todos os seus atributos não-chave dependem funcionalemente da chave primária.

![2NF](Screenshot_11.png)

![2NF](Screenshot_12.png)

#### 3NF - Terceira Forma Normal

Uma tabela está na 3FN quando estiver na 2FN e se todos os atributos que não pertencem à chave primária não dependerem de outros atributos que não pertencem à chave primária.

![3NF](Screenshot_13.png)

![3NF](Screenshot_14.png)

![3NF](Screenshot_15.png)

![3NF](Screenshot_16.png)

## Aula 16 - Transactions

Para garantir que o banco de dados MySQL não está operando em modo *auto-commit* use o comando `START TRANSACTION`. Este comando garante que estamos iniciando uma transação sem *auto-commit*. Por exemplo:

```sql	
START TRANSACTION;
INSERT INTO usuario (nome, sobrenome) VALUES ('João', 'Silva');
COMMIT
```

Se apenas realizassemos o `INSERT` sem o `COMMIT`, o banco de dados não iria salvar as alterações. O `COMMIT` é o comando que efetivamente salva as alterações no banco de dados.

```sql
START TRANSACTION;

SELECT id_usuario INTO @id_usuario 
FROM usuario 
WHERE nome = 'João' AND sobrenome = 'Silva'
ORDER BY id_usuario DESC LIMIT 1;

DELETE FROM usuario WHERE id_usuario=@id_usuario
COMMIT
```

O comando `ROLLBACK` é utilizado para desfazer as alterações
    
```sql
INSERT INTO usuario (nome, sobrenome) VALUES ('Walter', 'Branco');
```

Isso devolveria, por exemplo:

Executando query:

(1, 'Juca', 'Silva', Decimal('0.00'))

(2, 'Mario', 'Ferreira', Decimal('0.00'))

(3, 'Ana', 'Soares', Decimal('0.00'))

(4, 'Antonio', 'Reis', Decimal('0.00'))

(5, 'Paulo', 'Oliveira', Decimal('0.00'))

(6, 'Carla', 'Nakamura', Decimal('0.00'))

(7, 'Maria', 'Fontana', Decimal('0.00'))

(9, 'Walter', 'Branco', Decimal('0.00'))

Se fizermos um `ROLLBACK`:

```sql	
ROLLBACK;
```

Executando query:

(1, 'Juca', 'Silva', Decimal('0.00'))

(2, 'Mario', 'Ferreira', Decimal('0.00'))

(3, 'Ana', 'Soares', Decimal('0.00'))

(4, 'Antonio', 'Reis', Decimal('0.00'))

(5, 'Paulo', 'Oliveira', Decimal('0.00'))

(6, 'Carla', 'Nakamura', Decimal('0.00'))

(7, 'Maria', 'Fontana', Decimal('0.00'))

    

### Aula 17 - ACID

**ACID** é um acrônimo para Atomicidade, Consistência, Isolamento e Durabilidade.

**Atomicidade**: Todas as operações de uma transação devem ser executadas com sucesso, caso contrário, a transação deve ser desfeita.

 Exercício: descreva uma situação onde várias operações de banco de dados devem ser executadas atomicamente.

    R: Transferência de dinheiro entre contas bancárias.

    No caso de uma transação de transferência de dinheiro entre duas contas, a operação de crédito na conta do destinatário depende diretamente do sucesso da operação de débito na conta do remetente. Se a operação de débito falha — por exemplo, por falta de fundos suficientes —, a operação de crédito na conta do destinatário não será sequer iniciada.

**Consistência**: O banco de dados passa de um estado válido para outro estado válido. Ou seja, o banco de dados deve ser consistente antes e depois da transação.

    Exercício: ON DELETE pode ajudar a manter a consistência em deletes físicos. Como você pode garantir a consistência em deletes lógicos?

    R: Através de triggers, que são procedimentos armazenados que são executados automaticamente em resposta a determinados eventos em uma tabela ou exibição. Por exemplo, ao excluir um registro de uma tabela, um trigger pode ser acionado para excluir registros relacionados em outra tabela.

**Isolamento**: As transações devem ser isoladas umas das outras. Ou seja, uma transação não deve interferir em outra transação.

    Exercício: descreva uma situação onde a falta de isolamento pode causar problemas.

    R: Suponha que duas transações estejam tentando acessar a mesma linha de uma tabela ao mesmo tempo. Se a primeira transação não tiver sido concluída e a segunda transação tentar acessar a mesma linha, ela poderá acessar dados inconsistentes.

- 4 níveis de isolamento:

    - SERIALIZABLE → mais restritivo, uma transação que tente atualizar dados não-gravados por outra transação será rejeitada.

    - REPETABLE READ → se você leu algumas linhas e tenta ler elas novamente, os mesmos valores serão retornados.

    - READ COMMITED → se uma transação ocorrer em paralelo com a nossa transação e fizer o commit de seus dados, então nossa transação pode acabar lendo dados de linha modificados.

    - READ UNCOMMITED → um dados modificado de uma transação não-finalizada será acessível pela nossa transação (*dirty read*). É o nível de isolamento mais periogoso.


![ACID](Screenshot_17.png)

**Durabilidade**: quando uma transação é confirmada (COMMIT), ela permanecerá gravada mesmo que ocorra uma falha no sistema ou a energia seja desligada.

### Aula 18 - Functions, procedures e triggers

##### Stored procedures

São blocos de código que são armazenados no banco de dados e podem ser chamados por outros programas.

**Vantagens**:

- Reuso de código → definir tarefas rotineiras e reutilizá-las em diferentes partes do programa.
- Segurança → podemos bloquear todos os acessos a tabelas e definir apenas o acesso às stored procedures. Assim, ao invés de por exemplo, permitir acesso à tabela *usuário*, podemos permitir acesso ao procedimento `saque()` ou `consulta_saldo()`.
- Desempenho → ao invés de mandar vários comandos SQL para a base de dados, podemos chamar apenas um procedimento, pois todos os comandos SQL estão dentro do procedimento. Ademais, a base de dados não precisa compilar o procedimento toda vez que este é chamado.

**Desvantagens**:

- Debugging → pode ser difícil de debugar um procedimento armazenado.
- Portabilidade → a sintaxe de definição de procedimentos armazenados varia de um banco de dados para outro.
- Separação entre dados e lógica de negócio → com procedimentos armazenados estamos migrando parte da lógica de negócio para o banco de dados. Alterações na lógica de negócio agora demandam modificações e manutenção em partes distintas do sistema.
- Aumento da carga de processamento no servidor → servidores de banco de dados costumam ser otimizados para memória e largura de banda, não para processamento.




##### Criando uma stored procedure

```sql
USE emprestimos;

DROP PROCEDURE IF EXISTS adiciona_usuario;

CREATE PROCEDURE adiciona_usuario(IN novo_nome VARCHAR(80), IN novo_sobrenome VARCHAR(80))
BEGIN
    INSERT INTO usuario (nome, sobrenome)
    VALUES (novo_nome, novo_sobrenome);
END;
```

Assim, quando quisermos adicionar um usuário, basta chamar a stored procedure `adiciona_usuario()`.

```sql
START TRANSACTION
try:
    CALL adiciona_usuario('Juca', 'Silva');
    CALL adiciona_usuario('Mario', 'Ferreira');
    CALL adiciona_usuario('Ana', 'Soares');
    CALL adiciona_usuario('Antonio', 'Reis');
    CALL adiciona_usuario('Paulo', 'Oliveira');
    COMMIT;
except Exception as e:
    print(e)
    ROLLBACK
```


##### Stored functions

São blocos de código que são armazenados no banco de dados e podem ser chamados por outros programas.

Exemplo:
(versão mysql)
```sql
USE emprestimos;

DROP FUNCTION IF EXISTS saldo;

DELIMITER //
CREATE FUNCTION saldo(id INT) RETURNS DECIMAL(30, 2) READS SQL DATA
BEGIN
    DECLARE saldo_procurado DECIMAL(30, 2);

    SELECT 
        IFNULL(saldo, 0.0)
    INTO saldo_procurado FROM
        usuario
    WHERE
        id_usuario = id;

    RETURN saldo_procurado;
END//
DELIMITER ;
```

(versão python)
```sql
USE emprestimos;

DROP FUNCTION IF EXISTS saldo;

CREATE FUNCTION saldo(id INT) RETURNS DECIMAL(30, 2) READS SQL DATA
BEGIN
    DECLARE saldo_procurado DECIMAL(30, 2);

    SELECT 
        IFNULL(saldo, 0.0)
    INTO saldo_procurado FROM
        usuario
    WHERE
        id_usuario = id;

    RETURN saldo_procurado;
END;
```

Testando a função:

```sql
SELECT saldo(1);
```

##### Triggers

Procedimentos armazenados que são executados automaticamente quando uma operação é realizada.

Você pode escolher se o trigger ocorre antes ou depois da operação.

Exemplo:

```sql
USE emprestimos;

DROP TRIGGER IF EXISTS trig_movimentacao;

DELIMITER //
CREATE TRIGGER trig_movimentacao 
BEFORE INSERT ON movimentacao
FOR EACH ROW
BEGIN
    UPDATE usuario 
        SET saldo = saldo + NEW.valor 
        WHERE id_usuario = NEW.id_usuario;
END//

DELIMITER ;
```

Trigger para quando o saldo é insuficiente:

```sql
USE emprestimos;

DROP TRIGGER IF EXISTS trig_movimentacao;

DELIMITER //
CREATE TRIGGER trig_movimentacao 
BEFORE INSERT ON movimentacao
FOR EACH ROW
BEGIN
    UPDATE usuario 
        SET saldo = saldo + NEW.valor 
        WHERE id_usuario = NEW.id_usuario;
END//

DELIMITER ;
```

Já na versão atual do MySQL o check de saldo pode ser implementado normalmente com o constraint `CHECK`

```SQL
USE emprestimos;
ALTER TABLE usuario ADD CONSTRAINT chk_saldo CHECK (saldo >= 0.0);
ALTER TABLE emprestimo ADD CONSTRAINT chk_emprestimo CHECK (valor_atual >= 0.0);
```

##### Relembrando views

*Views* são como tabelas *'lógicas'*, que são criadas através da aplicação de um comando `SELECT`. É como um `SELECT` pré-armazenado.

Vamos criar uma *view* para listar os nomes e sobrenomes de usuários, sem revelar seus saldos de conta:

```SQL
USE emprestimos;

CREATE VIEW nomes AS 
	SELECT DISTINCT nome, sobrenome FROM usuario;
```

### Aula 20 - Programação Funcional

#### Cálculo Lambda

![Lambda](Screenshot_18.png)


![Lambda](Screenshot_19.png)

![Lambda](Screenshot_20.png)
![Lambda](Screenshot_21.png)

- Programação funcional descreve o que queremos calcular sem especificar como iterar sobre os dados

- Programação funcional usa funções puras:
    - Robustez → se um bloco de cálculo falha, podemos reiniciar o cálculo daquele bloco apenas.

- Ausência de estado global → facilita uso de memória distribuída

##### Funções anônimas (lambda)

Funções anônimas são funções que não possuem um nome associado. Normalmente usados quando precisamos de uma função simples.

Exemplo:

```python
def mapeia(func, vec):
    resultado = []
    for value in vec:
        resultado.append(func(value))
    return resultado

data = [2, 3, 5, 7]
res = mapeia(lambda x: x * x, data)
print(res)
```

Resultado:

```python
[4, 9, 25, 49]
```

Logo, a função `lambda x: x * x` é uma função anônima que recebe um argumento `x` e retorna `x * x`. Ela é passada como argumento para a função `mapeia()`, que aplica a função a cada elemento do vetor `data`, appendando o resultado em `resultado`.

##### Closures

Temos uma função definida dentro de outra função. Essa função interna usa variáveis do escopo da função externa. A função externa retorna a função interna.

Exemplo:

```python
def multiplicador(n):
    def multiplicador_de_x(x):
        return x * n
    return multiplicador_de_x

mult_2 = multiplicador(2)
mult_3 = multiplicador(3)

print(mult_2(5))
print(mult_3(5))
```

Resultado:

```python
10
15
```

A função `multiplicador()` retorna a função `multiplicador_de_x()`, que multiplica um número `x` por `n`. Assim, `mult_2` é uma função que multiplica por 2, e `mult_3` é uma função que multiplica por 3. Passando o argumento 5, temos 10 e 15, respectivamente.

##### List Comprehensions

List comprehensions são uma forma concisa de criar listas em Python. São usadas para criar novas listas onde cada elemento é o resultado de alguma operação aplicada a cada membro de outra sequência ou iterável.

Exemplo:

```python
data = [2, 3, 5, 7]
res = [x * x for x in data]
print(res)
```

Resultado:

```python
[4, 9, 25, 49]
```

Aqui, `res` é uma lista que contém o quadrado de cada elemento de `data`.

Outros exemplos:

```python
vec = list(range(20))
vec_even = [x for x in vec if x % 2 == 0]
print(vec_even)
```

Resultado:

```python
[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]
```

```python
res = [(x, y) for x in [1, 2, 3] for y in ["a", "b"]]
print(res)
```

Resultado:

```python
[(1, 'a'), (1, 'b'), (2, 'a'), (2, 'b'), (3, 'a'), (3, 'b')]
```



##### Higher-order functions

Funções que recebem outras funções como argumentos ou retornam funções.
Algumas das mais importantes são `map()`, `filter()` e `reduce()`.

- `map()` → recebe uma função e uma estrutura de dados iterável, e aplica a função a cada elemento da estrutura de dados. Retorna um objeto `map`, que é um iterador.

Exemplo:

```python
vec = [2, 3, 5]
aux = map(lambda x: x ** 2, vec)
print(aux)
print(list(aux))
```

Resultado:

```python
<map object at 0x7f8b1c3b3d90>
[4, 9, 25]
```

- `filter()` → recebe uma função e uma estrutura de dados iterável, e retorna um iterador com os elementos da estrutura de dados para os quais a função retorna `True`.

Exemplo:

```python
vec = list(range(10))
aux = filter(lambda x: x % 2 == 1, vec)
print(aux)
print(list(aux))
```

Resultado:

```python
<filter object at 0x7f8b1c3b3d90>
[1, 3, 5, 7, 9]
```

- `reduce()` → recebe uma função, uma estrutura de dados iterável e um valor inicial, e combina os valores do iterável usando a função que serve para combinar os valores dois a dois.

Exemplo:

```python
from functools import reduce

vec = list(range(10))
soma = reduce(lambda x, y: x + y, vec)
print(soma)
```

Resultado:

```python
45
```